# Algorithm

This is a Python implementation of Hankel temporal matrix factorization (HTMF).

In [ ]:
import numpy as np

def compute_mape(var, var_hat):
    return np.sum(np.abs(var - var_hat) / var) / var.shape[0]

def compute_rmse(var, var_hat):
    return np.sqrt(np.sum((var - var_hat) ** 2) / var.shape[0])

def update_cg(var, r, q, Aq, rold):
    alpha = rold / np.inner(q, Aq)
    var = var + alpha * q
    r = r - alpha * Aq
    rnew = np.inner(r, r)
    q = r + (rnew / rold) * q
    return var, r, q, rnew

def ell_w(ind, W, X, rho = 1e+1):
    return X @ ((W.T @ X) * ind).T + rho * W

def conj_grad_w(sparse_mat, ind, W, X, maxiter = 5):
    rank, dim1 = W.shape
    w = np.reshape(W, -1, order = 'F')
    r = np.reshape(X @ sparse_mat.T - ell_w(ind, W, X), -1, order = 'F')
    q = r.copy()
    rold = np.inner(r, r)
    for it in range(maxiter):
        Q = np.reshape(q, (rank, dim1), order = 'F')
        Aq = np.reshape(ell_w(ind, Q, X), -1, order = 'F')
        w, r, q, rold = update_cg(w, r, q, Aq, rold)
    return np.reshape(w, (rank, dim1), order = 'F')

def ell_x(ind, W, X, F, gamma, rho = 1e+1):
    return W @ ((W.T @ X) * ind) + rho * X + gamma * X

def conj_grad_x(sparse_mat, ind, W, X, F, gamma = 1e+3, maxiter = 5):
    rank, dim2 = X.shape
    x = np.reshape(X, -1, order = 'F')
    r = np.reshape(W @ sparse_mat + gamma * F - ell_x(ind, W, X, F, gamma), -1, order = 'F')
    q = r.copy()
    rold = np.inner(r, r)
    for it in range(maxiter):
        Q = np.reshape(q, (rank, dim2), order = 'F')
        Aq = np.reshape(ell_x(ind, W, Q, F, gamma), -1, order = 'F')
        x, r, q, rold = update_cg(x, r, q, Aq, rold)
    return np.reshape(x, (rank, dim2), order = 'F')

def hankel(mat, d):
    dim1, dim2 = mat.shape
    col_num = dim2 - d + 1
    hmat = np.zeros((dim1 * d, col_num))
    for k in range(d):
        hmat[k * dim1 : (k + 1) * dim1, :] = mat[:, k : col_num + k]
    return hmat

def inv_hankel(hmat, rank, d):
    dim1, dim2 = hmat.shape
    temp = np.zeros((rank, d, dim2 + d - 1))
    for k in range(d):
        temp[:, k, k : dim2 + k] = hmat[k * rank : (k + 1) * rank, :]
    return np.true_divide(temp.sum(1), (temp != 0).sum(1))

def hard_thresholding(hmat, rank):
    u, s, v = np.linalg.svd(hmat, full_matrices = False)
    return u[:, : rank] @ np.diag(s[: rank]) @ v[: rank, :], u[:, : rank], v[: rank, :].T

def hankel_tmf(dense_mat, sparse_mat, rank, d, maxiter):
    dim1, dim2 = sparse_mat.shape
    W = 0.01 * np.random.randn(rank, dim1)
    X = 0.01 * np.random.randn(rank, dim2)
    F = X.copy()
    if np.isnan(sparse_mat).any() == False:
        ind = sparse_mat != 0
        pos_test = np.where((dense_mat != 0) & (sparse_mat == 0))
    elif np.isnan(sparse_mat).any() == True:
        pos_test = np.where((dense_mat != 0) & (np.isnan(sparse_mat)))
        ind = ~np.isnan(sparse_mat)
        sparse_mat[np.isnan(sparse_mat)] = 0
    dense_test = dense_mat[pos_test]
    del dense_mat
    show_iter = 50
    for it in range(maxiter):
        W = conj_grad_w(sparse_mat, ind, W, X)
        X = conj_grad_x(sparse_mat, ind, W, X, F)
        hmat = hankel(X, d)
        mat, u, v = hard_thresholding(hmat, rank)
        F = inv_hankel(mat, rank, d)
        mat_hat = W.T @ X
        if (it + 1) % show_iter == 0:
            temp_hat = mat_hat[pos_test]
            print('Iter: {}'.format(it + 1))
            print('MAPE: {:.6}'.format(compute_mape(dense_test, temp_hat)))
            print('RMSE: {:.6}'.format(compute_rmse(dense_test, temp_hat)))
            print()
    return mat_hat, W, X, u

def hankel_tmf_dic(obs, W, X, d):
    dim1, dim2 = obs.shape
    rank = X.shape[0]
    if np.isnan(obs).any() == False:
        ind = obs != 0
    elif np.isnan(obs).any() == True:
        ind = ~np.isnan(obs)
        obs[np.isnan(obs)] = 0
    X = conj_grad_x(obs, ind, W, X, X, gamma = 0)
    return X

def ell_v(ind, U, V):
    return U.T @ ((U @ V) * ind)

def conj_grad_hankel(hmat, ind, U, maxiter = 5):
    dim1, dim2 = hmat.shape
    rank = U.shape[1]
    V = np.random.rand(rank, dim2)
    v = np.reshape(V, -1, order = 'F')
    r = np.reshape(U.T @ hmat - ell_v(ind, U, V), -1, order = 'F')
    q = r.copy()
    rold = np.inner(r, r)
    for it in range(maxiter):
        Q = np.reshape(q, (rank, dim2), order = 'F')
        Aq = np.reshape(ell_v(ind, U, Q), -1, order = 'F')
        v, r, q, rold = update_cg(v, r, q, Aq, rold)
    return np.reshape(v, (rank, dim2), order = 'F')

from ipywidgets import IntProgress
from IPython.display import display

def rolling4cast(dense_mat, sparse_mat, pred_step, delta, rank, d, maxiter):
    dim1, T = sparse_mat.shape
    start_time = T - pred_step
    max_count = int(np.ceil(pred_step / delta))
    mat_hat = np.zeros((dim1, max_count * delta))
    f = IntProgress(min = 0, max = max_count) # instantiate the bar
    display(f) # display the bar
    for t in range(max_count):
        if t == 0:
            _, W, X, U = hankel_tmf(dense_mat[:, : start_time], sparse_mat[:, : start_time], rank, d, maxiter)
        else:
            X = hankel_tmf_dic(sparse_mat[:, : start_time + t * delta], W, X_new, d)
        hmat = hankel(np.append(X, np.zeros((rank, delta)), axis = 1), d)
        X_new = inv_hankel(U @ conj_grad_hankel(hmat, hmat != 0, U), rank, d)
        mat_hat[:, t * delta : (t + 1) * delta] = W.T @ X_new[:, - delta :]
        f.value = t
    small_dense_mat = dense_mat[:, start_time : T]
    pos = np.where(small_dense_mat != 0)
    mape = compute_mape(small_dense_mat[pos], mat_hat[pos])
    rmse = compute_rmse(small_dense_mat[pos], mat_hat[pos])
    print('Prediction MAPE: {:.6}'.format(mape))
    print('Prediction RMSE: {:.6}'.format(rmse))
    print()
    return mat_hat, W, X

# Experiments

This is a numerical evaluation on the Seattle Uber movement speed dataset in 2019. Below is the traffic state forecasting with different time horizons `delta = 1, 2, 3, 4`.

In [ ]:
# import cupy as np
import numpy as np

dense_mat = np.load('../datasets/hourly_speed_mat_2019_1.npz')['arr_0']
for month in range(2, 4):
    dense_mat = np.append(dense_mat, np.load('../datasets/hourly_speed_mat_2019_{}.npz'.format(month))['arr_0'], axis = 1)
dense_mat = dense_mat[:, : 24 * 7 * 10]

delta = 3

import time
for rank in [60]: # rank of matrix factorization
    for week in [8, 7, 6, 5, 4]:
        for d in [24]: # order of VAR
            start = time.time()
            pred_step = 7 * 24
            maxiter = 50
            mat_hat, W, X = rolling4cast(dense_mat[:, 24 * 7 * (10 - week - 1) : 24 * 7 * 10],
                                         dense_mat[:, 24 * 7 * (10 - week - 1) : 24 * 7 * 10],
                                         pred_step, delta, rank, d, maxiter)
            print('week = {}'.format(week))
            print('rank R = {}'.format(rank))
            print('Order d = {}'.format(d))
            end = time.time()
            print('Running time: %d seconds'%(end - start))

Forecasting results of HTMF with different time horizons. Both ground truth data and HTMF forecasts are averaged over 741 road segments whose missing rates on the test set are less than 10%.

In [ ]:
import matplotlib.pyplot as plt

pos = np.where(np.sum(dense_mat[:, - 7 * 24 :] > 0,
                      axis = 1) > 0.9 * 7 * 24)
print(len(pos[0]))
mat1 = dense_mat[pos[0], :]
mat2 = mat_hat[pos[0], :]

plt.rcParams['font.size'] = 12
fig = plt.figure(figsize = (8, 2))
ax = fig.add_subplot(1, 1, 1)
mat = np.zeros(7 * 24)
for i in range(7 * 24):
    pos = np.where(mat1[:, - 7 * 24 + i] > 0)
    mat[i] = np.mean(mat1[pos[0], - 7 * 24 + i])
# plt.plot(np.asnumpy(np.where(mat > 0)[0]),
#          np.asnumpy(mat[mat > 0]), 'o', markersize = 6.5, color = 'orange',
plt.plot((np.where(mat > 0)[0]),
         (mat[mat > 0]), 'o', markersize = 6.5, color = 'orange',
         markeredgecolor = 'gray',
         alpha = 0.75, label = 'Ground truth')
# plt.plot(np.asnumpy(np.mean(mat2, axis = 0)),
plt.plot((np.mean(mat2, axis = 0)),
         color = 'red', linewidth = 3,
         alpha = 0.8, label = 'HTMF')
# plt.xticks(np.asnumpy(np.arange(0, 24 * 7 + 1, 1 * 24)))
plt.xticks((np.arange(0, 24 * 7 + 1, 1 * 24)))
plt.xlabel('Time (hour)')
plt.ylabel('Speed (mph)')
plt.grid(axis = 'both', linestyle='dashed', linewidth = 0.1, color = 'gray')
ax.legend()
ax.tick_params(direction = "in")
ax.set_xlim([-1, 7 * 24])
ax.set_ylim([25, 55])
plt.show()
fig.savefig("forecasts_delta_{}.pdf".format(delta), bbox_inches = "tight")

Forecasting results of HTMF with `delta = 4`. These 14 example time series correspond to 14 road segments. The orange scatters indicate the observed speed values, while the red curves indicate the forecasts achieved by HTMF.

In [ ]:
import matplotlib.pyplot as plt

plt.rcParams['font.size'] = 12
fig = plt.figure(figsize = (10, 8))
for loc in range(14):
    ax = fig.add_subplot(7, 2, loc + 1)
    # plt.plot(np.asnumpy(dense_mat[loc, - 7 * 24 :]), 'o', markersize = 6.5,
    plt.plot((dense_mat[loc, - 7 * 24 :]), 'o', markersize = 6.5,
             color = 'orange', markeredgecolor = 'gray', alpha = 0.75, label = 'Ground truth')
    # plt.plot(np.asnumpy(mat_hat[loc, :]), color = 'red',
    plt.plot((mat_hat[loc, :]), color = 'red',
             linewidth = 2.5, alpha = 0.7, label = 'HTMF')
    # plt.xticks(np.asnumpy(np.arange(0, 24 * 7 + 1, 1 * 24)))
    plt.xticks((np.arange(0, 24 * 7 + 1, 1 * 24)))
    if loc == 12 or loc == 13:
        plt.xlabel('Time (hour)')
    if loc == 6 or loc == 7:
        plt.ylabel('Speed (mph)')
    plt.grid(axis = 'both', linestyle='dashed', linewidth = 0.1, color = 'gray')
    ax.tick_params(direction = "in")
    ax.set_xlim([-1, 7 * 24])
    if loc < 12:
        plt.xticks(color = 'w')
    if loc == 12 or loc == 4:
        ax.set_ylim(bottom = 25)
    elif loc == 2 or loc == 3 or loc == 5:
        ax.set_ylim(bottom = 20)
    elif loc == 9:
        ax.set_ylim(bottom = 5)
    else:
        ax.set_ylim(bottom = 10)
plt.show()
fig.savefig("htmf_forecasts_delta_{}.pdf".format(delta), bbox_inches = "tight")

The histogram of ground truth data and forecasts achieved by HTMF with `delta = 4` in the test set. The missing rate implies the ratio of missing values of road segments in the test set.

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

for k in range(1, 11):
    if k == 1:
        pos = np.where(np.sum(dense_mat[:, - 7 * 24 :] == 0, axis = 1) <= 0.1 * 7 * 24)
    elif k == 10:
        pos = np.where(np.sum(dense_mat[:, - 7 * 24 :] == 0, axis = 1) > 0.9 * 7 * 24)
    else:
        pos = np.where((np.sum(dense_mat[:, - 7 * 24 :] == 0, axis = 1) > 0.1 * (k - 1) * 7 * 24)
                       & (np.sum(dense_mat[:, - 7 * 24 :] == 0, axis = 1) <= 0.1 * k * 7 * 24))
    print(len(pos[0]))
    mat1 = dense_mat[pos[0], - 7 * 24 :]
    mat2 = mat_hat[pos[0], :]
    pos_new = np.where(mat1 > 0)

    plt.rcParams['font.size'] = 12
    fig = plt.figure(figsize = (4, 2))
    ax = fig.add_subplot(1, 1, 1)

    sns.distplot(mat1[pos_new], hist = False, kde = True,
                  kde_kws = {'linewidth': 3, 'color': 'orange', 'alpha': 0.65},
                  label = 'Ground truth')
    sns.distplot(mat2[pos_new], hist = False, kde = True,
                  kde_kws = {'linewidth': 3, 'color': 'red', 'alpha': 0.65},
                  label = 'HTMF')
    plt.tick_params(direction = "in")
    plt.xlim([0, 80])
    ax.legend(['Ground truth', 'HTMF'])
    plt.xlabel('Speed (mph)')
    plt.ylabel('Probability')
    plt.savefig("htmf_forecasts_hist_{}.pdf".format(k), bbox_inches = "tight")
    plt.show()

The variations of HTMF's loss functions with the iterations on different datasets.

In [ ]:
def hankel_tmf(dense_mat, sparse_mat, rank, d, maxiter,
               rho = 1e+1, gamma = 1e+3):
    dim1, dim2 = sparse_mat.shape
    W = 0.01 * np.random.randn(rank, dim1)
    X = 0.01 * np.random.randn(rank, dim2)
    F = X.copy()
    if np.isnan(sparse_mat).any() == False:
        ind = sparse_mat != 0
        pos_test = np.where((dense_mat != 0) & (sparse_mat == 0))
    elif np.isnan(sparse_mat).any() == True:
        pos_test = np.where((dense_mat != 0) & (np.isnan(sparse_mat)))
        ind = ~np.isnan(sparse_mat)
        sparse_mat[np.isnan(sparse_mat)] = 0
    dense_test = dense_mat[pos_test]
    del dense_mat
    show_iter = 50
    obj = np.zeros(maxiter)
    for it in range(maxiter):
        W = conj_grad_w(sparse_mat, ind, W, X)
        X = conj_grad_x(sparse_mat, ind, W, X, F)
        hmat = hankel(X, d)
        mat, u, v = hard_thresholding(hmat, rank)
        F = inv_hankel(mat, rank, d)
        mat_hat = W.T @ X
        obj[it] = (0.5 * np.sum(((sparse_mat - mat_hat) * ind) ** 2)
                   + 0.5 * rho * (np.sum(W ** 2) + np.sum(X ** 2))
                   + 0.5 * gamma * np.sum((F - X) ** 2))
        if (it + 1) % show_iter == 0:
            temp_hat = mat_hat[pos_test]
            print('Iter: {}'.format(it + 1))
            print('MAPE: {:.6}'.format(compute_mape(dense_test, temp_hat)))
            print('RMSE: {:.6}'.format(compute_rmse(dense_test, temp_hat)))
            print()
    return mat_hat, W, X, u, obj

import cupy as np

dense_mat = np.load('../datasets/hourly_speed_mat_2019_1.npz')['arr_0']
for month in range(2, 4):
    dense_mat = np.append(dense_mat, np.load('../datasets/hourly_speed_mat_2019_{}.npz'.format(month))['arr_0'], axis = 1)
dense_mat = dense_mat[:, : 24 * 7 * 10]

import time
converg = []
for week in [9, 8, 7, 6, 5, 4]:
    for rank in [30, 40, 50, 60]:
        for d in [24]: # order of VAR
            start = time.time()
            pred_step = 7 * 24
            maxiter = 50
            mat_hat, W, X, u, obj = hankel_tmf(dense_mat[:, 24 * 7 * (10 - week - 1) : 24 * 7 * 9],
                                               dense_mat[:, 24 * 7 * (10 - week - 1) : 24 * 7 * 9],
                                               rank, d, maxiter)
            converg.append(obj)
            print('week = {}'.format(week))
            print('rank R = {}'.format(rank))
            print('Order d = {}'.format(d))
            end = time.time()
            print('Running time: %d seconds'%(end - start))

import matplotlib.pyplot as plt
plt.rcParams['font.size'] = 12
plt.rcParams['mathtext.fontset'] = 'cm'

i = 0
for week in [9, 8, 7, 6, 5, 4]:
    fig = plt.figure(figsize = (4, 2.8))
    ax = fig.add_subplot(1, 1, 1)
    for k in range(4):
        plt.plot(np.asnumpy(np.arange(1, 51, 1)),
                 np.asnumpy(converg[2 * i + k]),
                 linewidth = 2, alpha = 0.9)
    plt.xlim([0, 50])
    plt.yscale('log')
    plt.xlabel('Iteration')
    plt.ylabel(r'Loss function $f$')
    plt.legend([r'$R = 30$', r'$R = 40$', r'$R = 50$', r'$R = 60$'])
    ax.tick_params(direction = "in")
    ax.tick_params(which = 'minor', direction = "in")
    plt.show()
    fig.savefig("HTMF_loss_func_vs_iter_{}_weeks.pdf".format(week), bbox_inches = "tight")
    i += 1

### License

<div class="alert alert-block alert-danger">
<b>This work is released under the MIT license.</b>
</div>